**Tensorflow GPU**

In [2]:
!pip install tensorflow-gpu

     |████████████████████████████████| 394.3MB 41kB/s 


**GPU Type**

In [1]:
!nvidia-smi

Mon Mar  1 16:04:54 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**Tensorflow Import**

In [3]:
import tensorflow as tf


**Libraries (InceptionV3)**

In [5]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

**Image Size and Paths**

In [6]:
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/indoor/train'
valid_path = '/content/drive/MyDrive/indoor/test'

**Inception V3 with imagenet weights**


In [7]:
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

87916544/87910968 [==============================] - 1s 0us/step


In [8]:
for layer in inception.layers:
    layer.trainable = False

**Number of Classes**

In [10]:
folders = glob('/content/drive/MyDrive/indoor/train/*')
folders

['/content/drive/MyDrive/indoor/train/Diningroom',
 '/content/drive/MyDrive/indoor/train/Livingroom',
 '/content/drive/MyDrive/indoor/train/Bedroom']

In [11]:
x = Flatten()(inception.output)

***Activation softmax due to multiple categories*** 

In [12]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=inception.input, outputs=prediction)

**Summary**

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

**Adam Optimizer**

In [15]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

**Data Augmentation**

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [19]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/indoor/train',
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

Found 930 images belonging to 3 classes.


In [20]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/indoor/test',
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical')

Found 179 images belonging to 3 classes.


**Training**

In [22]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=25,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
59/59 [==============================] - 107s 2s/step - loss: 0.5966 - accuracy: 0.9559 - val_loss: 9.2783 - val_accuracy: 0.5810
Epoch 2/25
59/59 [==============================] - 105s 2s/step - loss: 0.7094 - accuracy: 0.9419 - val_loss: 7.0254 - val_accuracy: 0.7821
Epoch 3/25
59/59 [==============================] - 105s 2s/step - loss: 0.4357 - accuracy: 0.9570 - val_loss: 6.8216 - val_accuracy: 0.7654
Epoch 4/25
59/59 [==============================] - 106s 2s/step - loss: 0.5438 - accuracy: 0.9602 - val_loss: 7.1529 - val_accuracy: 0.7318
Epoch 5/25
59/59 [==============================] - 105s 2s/step - loss: 0.7058 - accuracy: 0.9559 - val_loss: 10.3508 - val_accuracy: 0.6480
Epoch 6/25
59/59 [==============================] - 105s 2s/step - loss: 0.4655 - accuracy: 0.9624 - val_loss: 6.2558 - val_accuracy: 0.7374
Epoch 7/25
59/59 [==============================] - 105s 2s/step - loss: 0.5604 - accuracy: 0.9613 - val_loss: 7.2130 - val_accuracy: 0.7654
Epoch 8/25
5

In [23]:
from tensorflow.keras.models import load_model

model.save('model_inception.h5')